In [1]:
import os

# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-2.4.7'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1,693 kB]
Hit:12 http://ppa.launchpad.net/gra

In [2]:

# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-10-12 02:33:12--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.44MB/s    in 0.2s    

2020-10-12 02:33:12 (5.44 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   10293599|R14LVB34Z2Z53I|B000YMOQZY|     262935067|Knockin' On Heave...|Digital_Music_Pur...|          5|            0|          0|   N|                Y|           favorites|Great  rendition....|2015-08-31 00:00:00|
|         US|    6087195|R2M0R9CWPC82LT|B00ISAEC80|     452318038|      

In [5]:
# Recreate vine_table
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14LVB34Z2Z53I|          5|            0|          0|   N|                Y|
|R2M0R9CWPC82LT|          5|            0|          0|   N|                Y|
| RQNQI2R3BM52A|          4|            0|          0|   N|                Y|
| RTUIX1135HUAB|          5|            0|          0|   N|                Y|
| RE3QSAT8TL010|          4|            0|          0|   N|                Y|
|R14TN65IY0NUOD|          5|            0|          0|   N|                Y|
|R3LIF8MCNH1ODF|          5|            0|          0|   N|                Y|
|R33X0DGA4PPQ3L|          5|            3|          3|   N|                N|
|R2DJJP7DQRT1ZW|          5|            0|          1|   N|                Y|
|R2A8XDXW5XOT4H|          5|            0|          0|   N|     

In [6]:
# 1. New DF for total_votes equal or greater than 20
votesmorethan20 = vine_df[vine_df["total_votes"] >= 20 ]
votesmorethan20.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1S0N56H5B9SUX|          5|           30|         30|   N|                Y|
| RCDI7Z8VCZJI9|          4|           40|         43|   N|                N|
|R3F4225VTUO2WE|          1|            8|         33|   N|                N|
|  R3ADPNURCCTK|          1|            1|         39|   N|                N|
|R2LWBYC8TEUVF2|          5|           50|         53|   N|                Y|
|R23E4EIJXID5IC|          5|           43|         43|   N|                N|
|R1WFSWDT8L5RSJ|          4|           23|         26|   N|                N|
|R37LMHN02KOER1|          5|           50|         50|   N|                N|
|R297QI4VU3LMY6|          4|           41|         50|   N|                N|
| RGNU1L1XFX33B|          3|           34|         35|   N|     

In [7]:
# 2. create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpfulvotesmorethan50 = votesmorethan20[votesmorethan20["helpful_votes"]/votesmorethan20["total_votes"] >= 0.5]
helpfulvotesmorethan50.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1S0N56H5B9SUX|          5|           30|         30|   N|                Y|
| RCDI7Z8VCZJI9|          4|           40|         43|   N|                N|
|R2LWBYC8TEUVF2|          5|           50|         53|   N|                Y|
|R23E4EIJXID5IC|          5|           43|         43|   N|                N|
|R1WFSWDT8L5RSJ|          4|           23|         26|   N|                N|
|R37LMHN02KOER1|          5|           50|         50|   N|                N|
|R297QI4VU3LMY6|          4|           41|         50|   N|                N|
| RGNU1L1XFX33B|          3|           34|         35|   N|                N|
|R2NPE00F48X0ET|          5|           22|         27|   N|                N|
| RSPQFPJ4ISKS3|          5|           32|         35|   N|     

In [9]:
# 3. create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
vine_Y = helpfulvotesmorethan50[helpfulvotesmorethan50['vine'] == 'Y' ]
vine_Y.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [10]:
# 4. Same as #3 but unpaid
vine_N = helpfulvotesmorethan50[helpfulvotesmorethan50['vine'] == 'N' ]
vine_N.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1S0N56H5B9SUX|          5|           30|         30|   N|                Y|
| RCDI7Z8VCZJI9|          4|           40|         43|   N|                N|
|R2LWBYC8TEUVF2|          5|           50|         53|   N|                Y|
|R23E4EIJXID5IC|          5|           43|         43|   N|                N|
|R1WFSWDT8L5RSJ|          4|           23|         26|   N|                N|
|R37LMHN02KOER1|          5|           50|         50|   N|                N|
|R297QI4VU3LMY6|          4|           41|         50|   N|                N|
| RGNU1L1XFX33B|          3|           34|         35|   N|                N|
|R2NPE00F48X0ET|          5|           22|         27|   N|                N|
| RSPQFPJ4ISKS3|          5|           32|         35|   N|     

In [11]:
#4. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

total_reviews = vine_df.count()
total_5starreviews_5 = vine_df[vine_df["star_rating"] == 5].count()
total_paid  = vine_Y.count()
total_unpaid = vine_N.count()
total_5star_paid = vine_Y[vine_Y["star_rating"] == 5].count()
total_5star_unpaid = vine_N[vine_N["star_rating"] == 5].count()
percentage_5star_paid = total_5star_paid/total_paid
percentage_5star_unpaid = total_5star_unpaid/total_unpaid



ZeroDivisionError: ignored

In [12]:
#4. Modified above since there are no paid vines, therefore cannot divide by 0
total_reviews = vine_df.count()
total_5starreviews_5 = vine_df[vine_df["star_rating"] == 5].count()
total_paid  = vine_Y.count()
total_unpaid = vine_N.count()
total_5star_paid = vine_Y[vine_Y["star_rating"] == 5].count()
total_5star_unpaid = vine_N[vine_N["star_rating"] == 5].count()
percentage_5star_paid = 0
percentage_5star_unpaid = total_5star_unpaid/total_unpaid

In [13]:
print(f"Total reviews = {total_reviews}")
print(f"Total 5 star reviews = {total_5starreviews_5}")
print(f"Total percentage unpaid = {percentage_5star_unpaid}")
print(f"Total percentage paid = {percentage_5star_paid}")

Total reviews = 1688884
Total 5 star reviews = 1345146
Total percentage unpaid = 0.5531774051191527
Total percentage paid = 0
